Ridge regression with splines and interactions on MDA_dataframeArthur.csv

Time Series cross-validation: 24h delay for weather, 1 month gap between train and test sets.
MAE en RMSE can be compared to other models.

In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_validate
import sklearn.model_selection
from sklearn import*
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import SplineTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import FeatureUnion

Dataframe from MDA_merge_data.ipynb is loaded, datetime column is dropped in order to run model.

In [2]:
df = pd.read_csv("C:\\Users\Arthur\hello\MDA_dataframeArthurKMI.csv")
df_nd = df.drop('result_timestamp', axis=1)

The target dataset (y) and predictor dataset (X) are created. For y, noise is converted to relative noise (proportion of max value)

In [3]:
y = df_nd["laeq"] / df_nd["laeq"].max()
X = df_nd.drop("laeq", axis="columns")

In [4]:
#create time-sensitive split for cross-validation
ts_cv = TimeSeriesSplit(
    n_splits=5,
    gap=4320,
    max_train_size=10000,
    test_size=1000,
)

In [5]:
#inspect splits
all_splits = list(ts_cv.split(X, y))
train_0, test_0 = all_splits[0]
X.iloc[test_0]
X.iloc[train_0]

,hour,day,day_of_month,month,weekend,LC_HUMIDITY,LC_DWPTEMP,LC_n,LC_RAD,LC_RAININ,...,LC_TEMP_QCL0,LC_TEMP_QCL1,LC_TEMP_QCL2,LC_TEMP_QCL3,air_pressure,air_temperature,relative_humidity,precipitation,wind_speed,wind_direction
30922,11,4,5,8,0,71.0,19.28,32.0,257.0,0.0,...,24.78,24.78,24.723,23.846788,1013.26,23.45,63.79,0.0,5.40,305.3
30923,11,4,5,8,0,70.0,19.13,32.0,240.0,0.0,...,24.84,24.84,24.783,24.193493,1013.46,23.15,63.22,0.0,5.22,299.2
30924,11,4,5,8,0,70.0,19.06,31.0,234.0,0.0,...,24.78,24.78,24.723,24.126774,1013.59,23.04,63.18,0.0,5.61,304.6
30925,11,4,5,8,0,69.0,18.84,32.0,229.0,0.0,...,24.87,24.87,24.813,24.144667,1013.72,23.25,62.01,0.0,5.55,298.4
30926,12,4,5,8,0,68.0,18.81,32.0,191.0,0.0,...,24.99,24.99,24.933,24.224405,1013.72,23.31,62.39,0.0,5.65,302.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40917,2,4,28,10,0,91.0,13.00,38.0,0.0,0.0,...,14.39,14.39,14.333,14.198870,1017.68,16.55,80.25,0.0,6.88,165.7
40918,2,4,28,10,0,90.0,12.83,37.0,0.0,0.0,...,14.39,14.39,14.333,14.185340,1017.66,16.68,80.08,0.0,6.99,166.3
40919,2,4,28,10,0,90.0,12.78,38.0,0.0,0.0,...,14.39,14.39,14.333,14.206740,1017.68,16.56,79.75,0.0,6.42,165.4
40920,2,4,28,10,0,90.0,12.78,37.0,0.0,0.0,...,14.39,14.39,14.333,14.314940,1017.44,16.64,79.61,0.0,7.02,167.1


In [6]:
train_4, test_4 = all_splits[4]
X.iloc[test_4]
X.iloc[train_4]

,hour,day,day_of_month,month,weekend,LC_HUMIDITY,LC_DWPTEMP,LC_n,LC_RAD,LC_RAININ,...,LC_TEMP_QCL0,LC_TEMP_QCL1,LC_TEMP_QCL2,LC_TEMP_QCL3,air_pressure,air_temperature,relative_humidity,precipitation,wind_speed,wind_direction
34922,9,4,16,9,0,84.0,13.90,32.0,163.0,0.0,...,16.59,16.59,16.533,16.339917,1008.95,17.77,69.49,0.00,8.11,359.7
34923,9,4,16,9,0,82.0,13.60,32.0,210.0,0.0,...,16.78,16.78,16.723,16.384344,1008.97,17.63,70.59,0.00,8.05,359.5
34924,9,4,16,9,0,79.0,13.46,31.0,165.0,0.0,...,17.07,17.07,17.013,16.527561,1009.02,17.61,71.20,0.00,8.94,353.8
34925,9,4,16,9,0,79.0,13.63,32.0,144.0,0.0,...,17.31,17.31,17.253,16.762182,1009.04,17.53,71.46,0.00,8.68,353.1
34926,9,4,16,9,0,80.0,14.00,32.0,142.0,0.0,...,17.49,17.49,17.433,16.968314,1009.04,17.66,71.54,0.00,8.26,352.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44917,23,3,24,11,0,86.0,6.88,37.0,0.0,0.0,...,9.11,9.11,9.053,9.123540,999.67,9.50,86.47,0.00,9.36,192.7
44918,23,3,24,11,0,86.0,6.92,38.0,0.0,0.0,...,9.17,9.17,9.113,9.154146,999.64,9.53,86.57,0.03,8.66,191.4
44919,23,3,24,11,0,85.0,6.81,37.0,0.0,0.0,...,9.22,9.22,9.163,9.262167,999.61,9.50,86.58,0.00,8.84,186.4
44920,23,3,24,11,0,85.0,6.78,38.0,0.0,0.0,...,9.22,9.22,9.163,9.232913,999.91,9.45,86.33,0.00,9.20,191.1


In [7]:
def evaluate(model, X, y, cv):
    cv_results = cross_validate(
        model,
        X,
        y,
        cv=cv,
        scoring=["neg_mean_absolute_error", "neg_root_mean_squared_error"],
    )
    mae = -cv_results["test_neg_mean_absolute_error"]
    rmse = -cv_results["test_neg_root_mean_squared_error"]
    print(
        f"Mean Absolute Error:     {mae.mean():.3f} +/- {mae.std():.3f}\n"
        f"Root Mean Squared Error: {rmse.mean():.3f} +/- {rmse.std():.3f}"
    )


In [8]:
min_max_scaler=MinMaxScaler()
numerical_columns=['hour','day','day_of_month','month','weekend','LC_HUMIDITY','LC_DWPTEMP','LC_n',
                   'LC_RAD','LC_RAININ','LC_DAILYRAIN','LC_WINDDIR','LC_WINDSPEED','LC_RAD60','LC_TEMP_QCL0',
                   'LC_TEMP_QCL1','LC_TEMP_QCL2','LC_TEMP_QCL3']
alphas = np.logspace(-6, 6, 25)
naive_linear_pipeline = make_pipeline(
    ColumnTransformer(
        transformers=[
            ("minmax", min_max_scaler, numerical_columns),
        ],remainder="passthrough",
    ),
    RidgeCV(alphas=alphas),
)
evaluate(naive_linear_pipeline, X, y, cv=ts_cv)

Mean Absolute Error:     0.071 +/- 0.013
Root Mean Squared Error: 0.087 +/- 0.013


In [9]:
def periodic_spline_transformer(period, n_splines=None, degree=3):
    if n_splines is None:
        n_splines = period
    n_knots = n_splines + 1  # periodic and include_bias is True
    return SplineTransformer(
        degree=degree,
        n_knots=n_knots,
        knots=np.linspace(0, period, n_knots).reshape(n_knots, 1),
        extrapolation="periodic",
        include_bias=True,
    )
cyclic_spline_transformer = ColumnTransformer(
    transformers=[
        ("cyclic_month", periodic_spline_transformer(12, n_splines=6), ["month"]),
        ("cyclic_weekday", periodic_spline_transformer(7, n_splines=3), ["day"]),
        ("cyclic_hour", periodic_spline_transformer(24, n_splines=12), ["hour"]),
        ("cyclic_monthday", periodic_spline_transformer(30, n_splines=15), ["day_of_month"])
    ],
    remainder=MinMaxScaler(),
)
cyclic_spline_linear_pipeline = make_pipeline(
    cyclic_spline_transformer,
    RidgeCV(alphas=alphas),
)
evaluate(cyclic_spline_linear_pipeline, X, y, cv=ts_cv)

Mean Absolute Error:     0.048 +/- 0.009
Root Mean Squared Error: 0.058 +/- 0.008


In [10]:
hour_weekend_interaction = make_pipeline(
    ColumnTransformer(
        [
            ("cyclic_hour", periodic_spline_transformer(24, n_splines=8), ["hour"]),
            ("weekend", FunctionTransformer(lambda x: x == 1), ["weekend"]),
        ]
    ),
    PolynomialFeatures(degree=2, interaction_only=True, include_bias=False),
)

In [11]:
#adding combining the spline pipeline and the interactions pipeline
cyclic_spline_interactions_pipeline = make_pipeline(
    FeatureUnion(
        [
            ("marginal", cyclic_spline_transformer),
            ("interactions1", hour_weekend_interaction),            
        ]
    ),
    RidgeCV(alphas=alphas),
)
evaluate(cyclic_spline_interactions_pipeline, X, y, cv=ts_cv)

Mean Absolute Error:     0.045 +/- 0.009
Root Mean Squared Error: 0.054 +/- 0.008


In [12]:
from sklearn.kernel_approximation import Nystroem

cyclic_spline_poly_pipeline = make_pipeline(
    cyclic_spline_transformer,
    Nystroem(kernel="poly", degree=2, n_components=70, random_state=0),
    RidgeCV(alphas=alphas),
)
evaluate(cyclic_spline_poly_pipeline, X, y, cv=ts_cv)

Mean Absolute Error:     0.050 +/- 0.017
Root Mean Squared Error: 0.060 +/- 0.017
